In [1]:
import pandas as pd
import sqlite3
import tqdm
import random

# Exercise

# TODO 1: Read all the data from bc.csv, how many rows is there ? Columns ?

# Ouverture de la base de données
df = pd.read_csv("data/Chap1/bc.csv")

# Nombre de lignes
print(len(df.axes[0]))

# Nombre de colonnes
print(len(df.axes[1]))

569
33


In [61]:
# TODO 2: Insert the bc.csv in the sqlite format.
conn = sqlite3.connect('data/bc.db')
c = conn.cursor()

In [62]:
#TODO 3: In the same db create a new table called tomatch with two variables id_ and rdm_float.
def create_table():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS tomatch(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
        conn.commit()
    except Exception as e:
        print(str(e))
        
create_table()

In [63]:
#TODO 4: In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a float random for each observation.
query = """INSERT INTO tomatch(id_,rdm_float) VALUES (?,random())"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,2)):
    values = (i,)
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        conn.commit()
        list_of_insertion = []

100%|█████████████████████████████| 4975000/4975000 [00:08<00:00, 621804.97it/s]


In [64]:
#TODO 5: Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.
c.execute("""SELECT avg(rdm_float), count(rdm_float), max(rdm_float), min(rdm_float), sum(rdm_float) FROM bc
             INNER JOIN tomatch ON tomatch.id_=bc.id""")

docs = c.fetchall()
print(docs)

[(1.4980062458408022e+17, 243, 9.172732098943238e+18, -9.189344035063669e+18, 3.640155177393149e+19)]


In [65]:
#TODO 6: How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?
c.execute("""SELECT count(radius_mean) FROM bc
             WHERE "radius_mean" > 15""")
docs2 = c.fetchall()
print(docs2)

c.execute("""SELECT count(texture_mean) FROM bc
             WHERE "radius_mean" > 15 AND "texture_mean" > 20""")
docs3 = c.fetchall()
print(docs3)

[(173,)]
[(106,)]


In [66]:
#TODO 7: For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups
c.execute("""SELECT diagnosis, round(avg(radius_mean),2) FROM bc GROUP BY diagnosis""")
docs4 = c.fetchall()
print(docs4)
print(round((docs4[1][1] - docs4[0][1]),2))

[('B', 12.15), ('M', 17.46)]
5.31


In [67]:
#TODO 8: Change the name of the column "diagnosis" to "label"
c.execute("""ALTER TABLE bc RENAME COLUMN diagnosis TO label""")
conn.commit()

In [68]:
#TODO 9: Add a column named "day" of type DATE.
c.execute("""ALTER TABLE bc ADD day DATE""")
conn.commit()

In [69]:
#TODO 10: Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)
c.execute("ALTER TABLE bc ADD COLUMN area_mean_integer INTEGER")
conn.commit()
c.execute("UPDATE bc SET area_mean_integer = CAST(area_mean AS INTEGER)")
conn.commit()

In [70]:
#TODO 11: create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.
c.execute("""CREATE TABLE IF NOT EXISTS tomatch2(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
conn.commit()

query = """INSERT INTO tomatch2(id_,rdm_float) VALUES (?,random())"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,10)):
    values = (i,)
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        conn.commit()
        list_of_insertion = []

100%|███████████████████████████████| 995000/995000 [00:01<00:00, 609477.40it/s]


In [71]:
#TODO 12: Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2.
c.execute("""SELECT * FROM bc
             INNER JOIN tomatch2 ON tomatch2.id_ = bc.id
             INNER JOIN tomatch ON tomatch.id_ = bc.id
             WHERE bc.radius_mean > 15
             AND tomatch.rdm_float > 0.5
             AND tomatch2.rdm_float > 0.5""")
docs5 = c.fetchall()
print(docs5)

[(879830, 'M', 17.01, 20.26, 109.7, 904.3, 0.08772, 0.07304, 0.0695, 0.0539, 0.2026, 0.05223, 0.5858, 0.8554, 4.106, 68.46, 0.005038, 0.01503, 0.01946, 0.01123, 0.02294, 0.002581, 19.8, 25.05, 130.0, 1210, 0.1111, 0.1486, 0.1932, 0.1096, 0.3275, 0.06469, None, None, 904, 879830, 1.8785959356047283e+18, 879830, 7.402616299114416e+18), (915460, 'M', 15.46, 23.95, 103.8, 731.3, 0.1183, 0.187, 0.203, 0.0852, 0.1807, 0.07083, 0.3331, 1.961, 2.937, 32.52, 0.009538, 0.0494, 0.06019, 0.02041, 0.02105, 0.006, 17.11, 36.33, 117.7, 909.4, 0.1732, 0.4967, 0.5911, 0.2163, 0.3013, 0.1067, None, None, 731, 915460, 2.0734103071453485e+17, 915460, 5.801732882903427e+18)]
